<a href="https://colab.research.google.com/github/qkrqhdud/GMB_Gemma2-/blob/main/GMB_Gemma2-%20/PPP/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit
!pip install torch
!pip install unsloth
!pip install bitsandbytes
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4

In [10]:
!pip uninstall bitsandbytes

!pip install bitsandbytes


Found existing installation: bitsandbytes 0.44.1
Uninstalling bitsandbytes-0.44.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/bitsandbytes-0.44.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/bitsandbytes/*
    /usr/local/lib/python3.10/dist-packages/tests/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/tests/slow/__init__.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/test_dpo_slow.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/test_sft_slow.py
    /usr/local/lib/python3.10/dist-packages/tests/slow/testing_constants.py
Proceed (Y/n)? y
  Successfully uninstalled bitsandbytes-0.44.1
  Using cached bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
Using cached bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl (122.4 MB)


In [1]:
from transformers import pipeline
from unsloth import FastLanguageModel
import re

def load_model():
    finetune_model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "yunho123/lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    )

    return finetune_model,tokenizer

def get_output(post_input,finetune_model,tokenizer):
  # 프롬프트
  post_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

  ### Input:
  {}

  ### Instruction:
  "Please rate the following blog post from 1 to 10 on each of the following category and provide a brief comment explaining your score for each criterion: "Topic Relevance", "Value Provision", "Storytelling", "Expertise","Conciseness","Paragraph Structure","Attention-Grabbing Opening", "Call-to-Action (CTA)","Balance of Professionalism and Personal Touch" and "Hashtag Usage"
   Please provide the output in the following format."

    "category": "Topic Relevance",
    "score": 5,
    "explanation": "comment",
    "improvement_suggestion": "comment"
    #next category
    "category": ,
    "score": ,
    "explanation": "comment",
    "improvement_suggestion": "comment"


  ### Response:
  {}"""
  inputs = tokenizer(
  [
      post_prompt.format(
          post_input,
          ""
      )
  ], return_tensors = "pt").to("cuda")

  outputs = tokenizer.batch_decode(finetune_model.generate(**inputs, max_new_tokens = 516, use_cache = True))

  response_start = outputs[0].find("### Response:")
  if response_start != -1:
    output = outputs[0][response_start+len("### Response:"):].strip()

  result = []
  matches = re.findall(r'"category":\s*"(.*?)",\s*\n\s*"score":\s*(.*?),\s*\n\s*"explanation":\s*"(.*?)",\s*\n\s*"improvement_suggestion":\s*"(.*?)"', output)

  for match in matches:
      category, score, explanation, improvement_suggestion = match
      result.append({"category": category, "score": int(score), "explanation": explanation, "improvement_suggestion": improvement_suggestion})


  # 결과 출력
  return result

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
import streamlit as st
import numpy as np
import plotly.graph_objects as go

model,tokenizer = load_model()
# 페이지를 Wide 모드로 설정
st.set_page_config(page_title="PPP - 소셜 미디어 게시물 인기도 예측기", layout="wide")

# 별점 계산을 위한 함수
def get_star_rating(score):
    full_stars = int(score)
    half_star = score - full_stars
    return "★" * full_stars + ("½" if half_star >= 0.5 else "") + "☆" * (5 - full_stars - (1 if half_star >= 0.5 else 0))

# 박스 형태 점수 표시 함수
def score_box(score):
    filled = "■" * score
    empty = "□" * (10 - score)
    return f"{filled}{empty}"

# 1단: 헤더 및 서비스 소개
st.title("PPP - 소셜 미디어 게시물 인기도 예측기")
st.subheader("당신의 게시물을 더 효과적으로 만들어 보세요!")

# 간단한 서비스 설명
st.info("""
    PPP를 사용하여 LinkedIn 게시물을 분석하고, 성과를 개선하기 위한 개인화된 피드백과
    예상 결과를 예측해드립니다.
""")

# 2단: 입력 및 스타일 선택
st.markdown("### LinkedIn 게시물 입력 및 스타일 선택:")

# 입력과 슬라이더를 4:1 비율로 배치
col1, col2 = st.columns([4, 1])

with col1:
    # 포스트 입력창의 높이를 3배로 설정
    post_input = st.text_area("게시물을 입력하세요",
                              placeholder="여기에 LinkedIn 게시물을 복사하여 붙여넣으세요...",
                              height=300)

with col2:
    st.markdown("#### 스타일 선택:")
    tone = st.slider("Tone (Official <-> Casual)", 0, 10, 5)
    purpose = st.slider("Purpose (Informative <-> Motivational)", 0, 10, 5)
    engagement = st.slider("Engagement (Engaging <-> Descriptive)", 0, 10, 5)

# 분석 버튼을 입력 창 바로 아래에 배치
analyze_button = st.button("게시물 분석")

# 3단: 분석 결과 처리
st.markdown("---")

if analyze_button:
    if post_input.strip() == "":
        st.error("분석을 위해 LinkedIn 게시물을 입력해주세요.")
    else:

        # 평가 결과를 처리하는 함수 호출
        results = process_evaluation_results(post_input,model,tokenizer)

        # 평균 점수 계산
        user_score_average = np.mean(results["scores"])
        dummy_average = 5.8  # 더미 사용자 평균 점수

        # 평균 점수와 사용자 평균 점수를 별점으로 변환
        user_star_rating = get_star_rating(user_score_average / 2)  # 10점 만점을 5점 만점으로 변환
        dummy_star_rating = get_star_rating(dummy_average / 2)

        # 평균 점수와 별점 표시
        st.markdown("### 📊 평균 점수")
        st.success(f"이 게시물의 평균 점수: **{user_score_average:.1f}** ({user_star_rating})")
        st.info(f"서비스를 이용한 사용자들의 평균 점수: **{dummy_average:.1f}** ({dummy_star_rating})")

        # 4단: 레이더 차트와 세부 분석을 나란히 표시
        st.markdown("### 분석 결과")
        col1, col2 = st.columns([1, 1])

        # 첫 번째 컬럼: 레이더 차트
        with col1:
            st.markdown("#### 스타일 개요")
            categories = results["categories"]
            scores = results["scores"]

            # Plotly로 레이더 차트 생성
            fig = go.Figure()
            fig.add_trace(go.Scatterpolar(
                r=scores + [scores[0]],  # 마지막 값을 첫번째 값으로 연결해 레이더 차트를 닫음
                theta=categories + [categories[0]],  # 카테고리의 시작과 끝을 연결
                fill='toself',
                line=dict(color='blue')
            ))

            fig.update_layout(
                polar=dict(
                    radialaxis=dict(visible=True, range=[0, 10])
                ),
                showlegend=False,
                template="plotly_white"  # 깔끔한 UI를 위한 Plotly 스타일
            )

            st.plotly_chart(fig)

        # 두 번째 컬럼: 세부 분석 (토글 형태로 표시)
        with col2:
            st.markdown("#### 세부 분석")
            for i, category in enumerate(results["categories"]):
                with st.expander(f"{category} -  {results['scores'][i]}/10"):
                    st.write(f"**설명:** {results['explanations'][i]}")
                    st.warning(f"**개선 제안:** {results['suggestions'][i]}")

        # 5단: 해시태그 추천 및 참조 필요 여부를 카드 형태로 좌우 분리
        st.markdown("### 추가 분석 정보")
        col1, col2 = st.columns(2)

        with col1:
            st.markdown("**해시태그 추천**")
            st.success(f"추천 해시태그: {', '.join(results['additional_analysis']['recommended_hashtags'])}")

        with col2:
            st.markdown("**참조 필요 여부**")
            st.info(f"참조 필요 여부: {'필요' if results['additional_analysis']['references_needed'] else '필요하지 않음'}")

        # 6단: 원본과 개선본 비교 및 강조
        st.markdown("### 원본과 개선본 비교")
        original_text = post_input
        improved_text = """
        여기에서 개선된 게시물 텍스트를 예로 추가할 수 있습니다.
        변경된 부분 또는 추가된 부분을 하이라이트해보세요. 예를 들어, 중요한 변경사항은 **굵게** 표시.
        """

        col1, col2 = st.columns(2)

        with col1:
            st.markdown("**원본**")
            st.write(original_text)

        with col2:
            st.markdown("**개선본**")
            st.write(improved_text)

        # 개선된 텍스트의 변경된 부분 강조 (예시로 굵게 표시)
        improved_highlighted = improved_text.replace("개선된", "**개선된**").replace("추가된 부분", "**추가된 부분**")
        st.markdown(f"**개선된 부분 강조**: {improved_highlighted}")
else:
    st.info("분석 버튼을 눌러 분석 결과를 확인하세요.")

==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/216M [00:00<?, ?B/s]

Unsloth 2024.9.post4 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.
2024-10-02 05:13:26.643 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 05:13:26.645 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 05:13:26.768 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-02 05:13:26.769 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 05:13:26.770 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 05:13:26.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-02 05:13:2